In [ ]:
!pip install pyspark

In [ ]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("MySparkSession").getOrCreate()

# Check if the SparkSession is created successfully
spark

In [ ]:
# import pandas as pd
from textblob import TextBlob
import nltk
from nltk.tokenize import sent_tokenize

In [ ]:
df_reviews = spark.read.csv('/content/drive/MyDrive/ProcessedCSV/final_review.csv', header=True, inferSchema=True)


In [ ]:
# reviews.info()
# tips.info()

In [ ]:
# reviews.head()


In [ ]:
# tips.head()

In [ ]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from textblob import TextBlob

# Function to determine sentiment
def get_sentiment(text):
    analysis = TextBlob(text)
    # Classify sentiment
    if analysis.sentiment.polarity > 0:
        return 'positive'
    elif analysis.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Create a UDF (User-Defined Function) from the get_sentiment function
sentiment_udf = udf(get_sentiment, StringType())

# Apply the UDF to the 'text' column
df_reviews = df_reviews.withColumn('sentiment', sentiment_udf(df_reviews['text']))

# Save the updated DataFrame to a new CSV file
df_reviews.write.csv('reviews_with_sentiment.csv', header=True, mode='overwrite')

print("Sentiment analysis completed and saved to 'reviews_with_sentiment.csv'.")


In [ ]:
# read and display csv file
df_reviews_with_sentiment = spark.read.csv('/content/reviews_with_sentiment.csv', header=True, inferSchema=True)
df_reviews_with_sentiment.show()

In [ ]:

# Drop the first row from the DataFrame
df_reviews_with_sentiment = df_reviews_with_sentiment.filter(df_reviews_with_sentiment.index != 0)

# Show the updated DataFrame
df_reviews_with_sentiment.show()

In [ ]:
# save CSV file in the 'ProcessedCSV' folder
df_reviews_with_sentiment.write.csv('/content/drive/MyDrive/ProcessedCSV/reviews_with_sentiment.csv', header=True, mode='overwrite')